In [2]:
import pandas as pd
import numpy as np
try:
    import cPickle 
except:
    import pickle as cPickle  
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from gensim.models import word2vec
import nltk.data
import re
import logging

In [4]:
with open('./duyet_data_train_w2v', 'rb') as f:
    data_train_w2v = cPickle.load(f)

In [8]:
list_stopwords = stopwords.words()
stemer = PorterStemmer()

def nltk_clean_skills(list_of_skill):
    result_skills = []
    for skill in list_of_skill:
        if not isinstance(skill, str): #and not isinstance(skill, unicode):
            continue
        skill_split = skill.split('_')
        skill_cleaned = []
        for skill_word in skill_split:
            if skill_word not in list_stopwords:
                skill_word = stemer.stem(skill_word)
                if len(skill_word) > 0:
                    skill_cleaned.append(skill_word)
        result_skill = '_'.join(skill_cleaned)
        if result_skill not in result_skills:
            result_skills.append(result_skill)
    return result_skills
    
nltk_clean_skills(['sql_queries' , 'sql_designer', 'sql_design', 'caad', 'design', 'piping'])

['sql_queri', 'sql_design', 'caad', 'design', 'pipe']

In [9]:
data_train_w2v_for_check = data_train_w2v
data_train_w2v = []
for line in data_train_w2v_for_check:
    line = nltk_clean_skills(line)
    if line not in data_train_w2v:
        data_train_w2v.append(line)
        
print (len(data_train_w2v))
data_train_w2v

7865


[['addm',
  'configur_manag_databa',
  'servicenow',
  'servic_manag',
  'itil',
  'ddm',
  'bmc_remedi',
  'issu_track',
  'configur',
  'knowledg_manag',
  'help_desk',
  'normal',
  'bppm',
  'patrol',
  'bmc_patrol',
  'event_manag',
  'system_manag',
  'perform_manag',
  'awr',
  'tkprof',
  'sql_trace',
  'data_pump',
  'oracl_dba',
  'oracl_grid_control',
  'statspack',
  'tde',
  'recoveri',
  'mef',
  'vldb',
  'nmon'],
 ['adaba',
  'natur',
  'energi',
  'rosco',
  'fiscal_manag',
  'jcl',
  '',
  'entirex',
  'jar',
  'webmethod',
  'root_cau_analysi',
  'cost_estim',
  'assembl_languag',
  'bal',
  'smp/e',
  'ipc',
  'tpf',
  'pl/i',
  'ibm_util',
  'rexx',
  'syncsort',
  'clist',
  'smf',
  'panvalet',
  'tso',
  'ca-7',
  'endevor',
  'ops/mv',
  'comput_oper',
  'spf'],
 ['acsl',
  'netbackup',
  'backup',
  'backup_administr',
  'vtl',
  'emc_avamar',
  'recoveri',
  'data_domain',
  'storage+',
  'emc_recoverpoint',
  'emc_vmax',
  'emc_isilon',
  '',
  'ndmp',
  'be

In [10]:
import multiprocessing

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)    

print ("Training model...")
model = word2vec.Word2Vec(data_train_w2v, 
          workers=multiprocessing.cpu_count(),  # Number of threads to run in parallel
          size=300, 
          min_count=1, 
          window=10, 
          sample = 1e-3,  # Downsample setting for frequent words
          iter=4,
          sg =1
    )
model.init_sims(replace=True)


2021-01-10 16:30:28,467 : INFO : collecting all words and their counts
2021-01-10 16:30:28,468 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-01-10 16:30:28,531 : INFO : collected 9920 word types from a corpus of 194928 raw words and 7865 sentences
2021-01-10 16:30:28,533 : INFO : Loading a fresh vocabulary
2021-01-10 16:30:28,609 : INFO : effective_min_count=1 retains 9920 unique words (100% of original 9920, drops 0)
2021-01-10 16:30:28,609 : INFO : effective_min_count=1 leaves 194928 word corpus (100% of original 194928, drops 0)
2021-01-10 16:30:28,642 : INFO : deleting the raw counts dictionary of 9920 items


Training model...


2021-01-10 16:30:28,644 : INFO : sample=0.001 downsamples 2 most-common words
2021-01-10 16:30:28,645 : INFO : downsampling leaves estimated 193330 word corpus (99.2% of prior 194928)
2021-01-10 16:30:28,686 : INFO : estimated required memory for 9920 words and 300 dimensions: 28768000 bytes
2021-01-10 16:30:28,687 : INFO : resetting layer weights
2021-01-10 16:30:30,566 : INFO : training model with 4 workers on 9920 vocabulary and 300 features, using sg=1 hs=0 sample=0.001 negative=5 window=10
2021-01-10 16:30:31,756 : INFO : EPOCH 1 - PROGRESS: at 64.06% examples, 109559 words/s, in_qsize 7, out_qsize 0
2021-01-10 16:30:31,948 : INFO : worker thread finished; awaiting finish of 3 more threads
2021-01-10 16:30:32,048 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-01-10 16:30:32,077 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-01-10 16:30:32,089 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-01-10 16:30:32,090

In [11]:
model.wv.save_word2vec_format('duyet_word2vec_skill_nltk.bin', binary=True)

2021-01-10 16:34:29,917 : INFO : storing 9920x300 projection weights into duyet_word2vec_skill_nltk.bin


# Testing

In [12]:
model.similar_by_word('machine_learning')

C:\Users\Dima\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  """Entry point for launching an IPython kernel.


KeyError: "word 'machine_learning' not in vocabulary"

In [13]:
model.similar_by_word('python')

C:\Users\Dima\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  """Entry point for launching an IPython kernel.


[('rubi', 0.9668879508972168),
 ('perl', 0.9646422266960144),
 ('cassandra', 0.9429396390914917),
 ('linux', 0.9415709972381592),
 ('softwar_develop', 0.9362480044364929),
 ('spark', 0.9355037808418274),
 ('hadoop', 0.9300486445426941),
 ('aw', 0.9298261404037476),
 ('big_data', 0.9287647008895874),
 ('postgresql', 0.9240198135375977)]

In [14]:
model.similar_by_word('css')

C:\Users\Dima\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  """Entry point for launching an IPython kernel.


[('html', 0.9850928783416748),
 ('cross-brow', 0.9848188161849976),
 ('html5', 0.9839492440223694),
 ('web_develop', 0.9819834232330322),
 ('javascript', 0.9763885140419006),
 ('ui', 0.9744644165039062),
 ('xhtml', 0.9726681113243103),
 ('web_standard', 0.970990777015686),
 ('mobil_web', 0.9692897796630859),
 ('drupal', 0.9680929183959961)]

In [15]:
model.similar_by_word('html')

C:\Users\Dima\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  """Entry point for launching an IPython kernel.


[('web_develop', 0.9853659868240356),
 ('css', 0.9850928783416748),
 ('cross-brow', 0.9816641211509705),
 ('xhtml', 0.978906512260437),
 ('adob_dreamweav', 0.974524736404419),
 ('unobtru_javascript', 0.974357008934021),
 ('drupal', 0.9736711978912354),
 ('web_standard', 0.9726221561431885),
 ('html5', 0.969694972038269),
 ('dhtml', 0.9690181612968445)]

In [ ]:
model.similar_by_word('html5')

In [ ]:
model.similar_by_word('bootstrap')

In [ ]:
model.similar_by_word('javascript')

In [ ]:
model.similar_by_word('nodejs')

In [ ]:
model.similar_by_word('node.js')

In [ ]:
model.similar_by_word('php')

In [ ]:
model.similar_by_word('c++')

In [ ]:
model.similar_by_word('web')

In [ ]:
model.similar_by_word('rails')

In [ ]:
model.similar_by_word('ruby')

In [ ]:
model.similar_by_word('mysql')

In [ ]:
model.similar_by_word('db2')

In [ ]:
model.similar_by_word('sql')

In [ ]:
model.similar_by_word('mssql')

In [ ]:
model.similar_by_word('db2')

In [ ]:
model.similar_by_word('html5')

In [ ]:
model.similar_by_word('oracle')

In [ ]:
model.similar_by_word('php5')

In [ ]:
model.similar_by_word('asp')

In [ ]:
model.similar_by_word('svm')

In [ ]:
model.similar_by_word('django')

In [ ]:
model.similar_by_word('mongodb')

In [ ]:
model.similar_by_word('mongo')

In [ ]:
model.similar_by_word('falcon')

In [ ]:
model.similar_by_word('express')

In [ ]:
model.similar_by_word('spark')

In [ ]:
model.similar_by_word('hadoop')

In [ ]:
model.similar_by_word('hive')

In [ ]:
model.similar_by_word('impala')

In [ ]:
model.similar_by_word('oozie')

In [ ]:
model.similar_by_word('apache2')

In [ ]:
model.similar_by_word('nginx')

In [ ]:
model.similar_by_word('rest')

In [ ]:
model.similar_by_word('.net')

In [ ]:
model.similar_by_word('perl')

In [ ]:
model.similar_by_word('unity')

In [ ]:
model.similar_by_word('3d')

In [ ]:
model.similar_by_word('wordpress')

In [ ]:
model.similar_by_word('jquery')

In [ ]:
model.similar_by_word('ajax')